In [2]:
import sys
sys.path.append("../")

In [6]:
import os 
import tensorflow as tf

from src.data.TextToToken.CustomCharacterToken import CustomCharacterToken
from src.data.DataObjects.StandardTextDataObject import E2EStandardTextObject
from src.data.DataLoaders import get_data_from_hgset

In [7]:
content_token = CustomCharacterToken(use_bookmark=False)

sequence_length = 100
batch_size = 64
buffer_size = 10000
embedding_dimension = 128
dense_dimension = 256

In [8]:
my_data_set = E2EStandardTextObject(text_sequencer=content_token, data_loader=get_data_from_hgset
                                    , sequence_lenth=sequence_length, set_name="tiny_shakespeare", sequence_len=sequence_length)

Using the latest cached version of the module from C:\Users\willi\.cache\huggingface\modules\datasets_modules\datasets\tiny_shakespeare\b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e (last modified on Thu Dec 28 11:36:59 2023) since it couldn't be found locally at tiny_shakespeare., or remotely on the Hugging Face Hub.
Found cached dataset tiny_shakespeare (C:/Users/willi/.cache/huggingface/datasets/tiny_shakespeare/default/1.0.0/b5b13969f09fe8707337f6cb296314fbe06960bd9a868dca39e713e163d27b5e)


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
vocab_size_shake = my_data_set.text_sequencer.get_vocab_size()
training_dataset = my_data_set.batch_and_shuffle(batch_size=batch_size,buffer_size=buffer_size)

In [12]:
test_example = [my_data_set.token_list[0]]

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Define the model architecture
def seq2seq_model(input_vocab_size, output_vocab_size, embedding_dim, hidden_units):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(hidden_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(output_vocab_size, embedding_dim)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding(decoder_inputs), initial_state=encoder_states)
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    return model

# Example usage
input_vocab_size = vocab_size_shake  # Adjust based on your dataset
output_vocab_size = vocab_size_shake  # Adjust based on your dataset
embedding_dim = 256
hidden_units = 512

model = seq2seq_model(input_vocab_size, output_vocab_size, embedding_dim, hidden_units)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, None, 256)            16640     ['input_1[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, None, 256)            16640     ['input_2[0][0]']             
                                                                                              

In [16]:
import numpy as np

def extract_encoder_decoder_models(model):
    encoder_inputs = model.input[0]  # input_1 in some cases
    encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1 in some cases
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = Model(encoder_inputs, encoder_states)

    decoder_state_input_h = Input(shape=(hidden_units,))
    decoder_state_input_c = Input(shape=(hidden_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_inputs = model.input[1]  # input_2 in some cases
    decoder_embedding_layer = model.layers[3]  # embedding_1 in some cases
    decoder_lstm = model.layers[4]  # lstm_2 in some cases
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_embedding_layer(decoder_inputs), initial_state=decoder_states_inputs)
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.layers[5]  # dense_1 in some cases
    decoder_outputs = decoder_dense(decoder_outputs)

    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    return encoder_model, decoder_model

# Example usage
# Assuming 'model' is your trained Seq2Seq model
encoder_model, decoder_model = extract_encoder_decoder_models(model)

# Now you can use the infer_sequence function
input_text = "Hello, how are you?"
encoded_text = infer_sequence(input_text, encoder_model, decoder_model, input_tokenizer, output_tokenizer, max_output_length=20)

print(f"Input: {input_text}")
print(f"Translated output: {encoded_text}")


TypeError: Cannot iterate over a Tensor with unknown first dimension.